In [26]:

import sys
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
from os.path import isfile,join
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
#import doc2vec_impl
import gensim

import FileProcess as fp



In [27]:
print (fp.window_size)
batch_size=10
embedding_filename = "glove_6B_200d.txt"
training_epochs=1
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]


2


In [28]:
dirpath = "corpus/fulltext/"
files = [f for f in os.listdir(dirpath) if isfile(join(dirpath, f))]



In [29]:
stop_words_list=set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

##########loading dev2vec=============
#doc2vec_impl.main();

d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec_models/doc2vec.model')
#start testing
#printing the vector of document at index 1 in docLabels
docvec = d2v_model.docvecs[files[0]]
print(docvec)


[-2.7762175  -0.45936382 -0.618449   -0.8609642   0.12776479  0.7008039
 -0.18298188 -0.3065114  -1.0207447   0.44748068 -0.5251434  -0.6478685
  0.07363328 -0.9913516  -0.08037782  0.3569275   0.04830581 -0.3157292
 -0.33566946 -1.8007975  -0.6615384  -0.45662943  0.35856095 -1.6224141
  1.7292947   0.01610192  0.15269798 -0.2633624  -0.42103472  0.4310425
 -0.8186927  -1.817319   -1.9041965  -0.8068781  -1.777631   -1.8509216
 -0.4281155   0.4977019   0.61602515  0.76634425  1.3259608  -0.31260923
 -1.3816838  -0.37780055 -1.1810374  -1.6388259   1.0663178   0.06534389
 -0.7727587  -0.11205402 -1.9782453   0.08237717  1.9480875  -1.8146058
 -1.3543537   1.3657124  -0.25756338  0.54493374 -0.6701863   0.32007536
  0.05760738  1.294205   -0.32358631 -0.9725704  -1.0321938  -0.39936873
  0.15072757 -0.7044433  -1.2028613  -0.57092947 -1.0215961   2.1046119
 -0.02798054 -0.17244416  0.8805654   0.5527702   1.502657   -2.0060904
  0.67571783  1.7193362   1.0933233  -1.2774023   0.8924701 

In [30]:
#
def read_data(file_name):
    with open(file_name, 'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count = {};
        word_index={};

        word2vector = {}
        count=0;
        for line in f:
            line_ = line.strip()  # Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word_index[words_Vec[0]]=count;
            count=count+1;

            word2vector[words_Vec[0]] = np.array(words_Vec[1:], dtype=float)
    print("Total Words in DataSet:", len(word_vocab))
    return word_vocab, word2vector, word_index

In [31]:
def load_embeddings(filename):
    dvocab, w2v,word_index  = read_data(filename);
    return w2v,word_index

In [32]:

embedding_array,word_index = load_embeddings(embedding_filename)
print("Loaded embeddings")

# words_data=pd.read_csv("words_data.csv");
# catchphrases_data=pd.read_csv("catch_data.csv")
print("reading data completed");

('Total Words in DataSet:', 400000)
Loaded embeddings
reading data completed


In [33]:
#sen2vec embeddings
def sen2vec(sentences):
    embed = hub.Module(module_url)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        sentenceEmbeddings = session.run(embed(sentences))    
        return sentenceEmbeddings

# sentence_embeddings = sen2Vec(['UWA was ordered to pay the costs of Dr Gra','Sirtex succeeded in its cross-claim against Dr'])

In [ ]:
print(word_index['unk'])


In [34]:
def get_statements(file):
    with open(dirpath + file, 'r',encoding="utf-8", errors='replace') as f:
        data=str(f.read());
        data=data.lower()
        data = data.replace("\"id=", "id=\"");
        data=data.replace("\n","")
        data=data.replace('".*?=.*?"', "",)
        data=data.replace("&","");
        xml = ET.fromstring(str(data))
        name=None;
        rows_list=[];
        catchphrases=[];
        sentences=[];
        for child in xml:
            if child.tag=="catchphrases":
                for catchphrase in child:
                    id=catchphrase.attrib.get("id")
                    #print(catchphrase.text)
                    #catchphrases.append({"file_id":file,"Name":name,"Id":id,"text":catchphrase.text})
                    catchphrases+=tokenizer.tokenize(catchphrase.text)
            if child.tag=="sentences":
                for sentence in child:
                    id = sentence.attrib.get("id")
                    #sentences.append({"file_id":file,"Name":name,"Id":id,"text":sentence.text})
                    sentences+=tokenizer.tokenize(sentence.text)
    
    sentences = [w for w in sentences if not w in stop_words_list] 
    catchphrases = [w for w in catchphrases if not w in stop_words_list]
    
    #sentences=[word.lower() for word in sentences if word.isalphanum()]
    #catchphrases=[word.lower() for word in catchphrases if word.isalphanum()]
    
    return sentences,catchphrases


    

In [35]:
def prepare_index_matrices(sentence_indexes):
    length=len(sentence_indexes);
    
    sentence_indexes=[word_index['unk'],word_index['unk']]+sentence_indexes+[word_index['unk'],word_index['unk']];    
    out_indices=[]
    for i in range(2,length):
        out_indices.append(sentence_indexes[i-2:i+3]);
        
    return out_indices;

def prepare_label_matrices(labels):
    length=len(labels);
    labels=[0,0]+labels+[0,0];
    out_labels=[];
    
    for i in range(2,length):
        out_labels.append(labels[i]);
        
    return out_labels;
    

In [36]:
def lookup_indexes_labels(sentences,catchphrases):
    
    sentence_indexes=[];
    labels=[];
    
    for word in sentences:
        if word in word_index:
            sentence_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            sentence_indexes.append(word_index['unk']);
            
    for word in sentences:
        if word in catchphrases:
            labels.append(1.0);
        else:
            #print("unknown word",word)
            labels.append(0.0);
            
    return sentence_indexes,labels;

In [37]:
def get_file_sentences(file_name):
    temp_data=full_data[(full_data.file_id==file_name) & full_data['Id'].str.startswith(('s'))]
    return list(temp_data.text.values)[:-1]

In [43]:

class CatchPhraseExtractor(object):

    def __init__(self, graph):
        self.total_loss=0.0
        self.build_graph(graph)

    def build_graph(self, graph):
        print("generating graph");

        with graph.as_default():
            
            #feed_dict = {self.doc_embedding: doc_embedding, self.phrases_indices: phrases_indices, self.phrase_labels: phrases_label_matrix,
            #self.sentence_indices:sentence_ids_to_indices, self.sentence_embeddings: sentence_embeddings}            
            
            
            self.phrases_indices = tf.placeholder(tf.int32, shape=[None,5])
            self.phrase_labels = tf.placeholder(tf.int32, shape=[None,1])
            W = tf.Variable(embedding_array, dtype=tf.float32) 
            phrase_tensor = tf.nn.embedding_lookup(W, self.phrases_indices)
            
            self.sentence_ids_to_indices = tf.placeholder(tf.int32, shape=[None,1])
            self.sentence_embeddings = tf.placeholder(tf.int32, shape=[None,200])
            sentence_tensor = tf.nn.embedding_lookup(self.sentence_embeddings, self.sentence_ids_to_indices)
                                                   
            self.doc_embedding = tf.placeholder(tf.int32, shape=[1,200])
            document_tensor = tf.convert_to_tensor(np.array([self.doc_embedding for _ in xrange(self.train_sentence_indices.shape[0])]))
                     
            phrase_tensor = tf.reshape(phrase_tensor, [tf.shape(phrase_tensor)[0], 1000])
            conv_filter = tf.Variable(tf.truncated_normal((1000, 200), stddev=0.1));
            word_tensor = tf.nn.relu(tf.matmul(sentence_vectors, conv_filter));
            
            stacked_tensor = tf.concat(values=[word_tensor, sentence_tensor, document_tensor], axis=1)
                                                   
            weights_input = tf.Variable(tf.random_normal((600, 100), stddev=0.2));
            weights_input_2 = tf.Variable(tf.random_normal((100, 1), stddev=0.2));
            
            bias1 = tf.Variable(tf.random_normal((1, 100), stddev=0.2));
            bias2 = tf.Variable(tf.random_normal((1, 1), stddev=0.2));
            
            self.sentence_values=self.MLP(stacked_tensor, weights_input, weights_input_2, bias1, bias2);
            
            prediction_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.train_labels,logits=self.sentence_values)
            self.loss=tf.reduce_mean(total_loss)
                               
            #using the gradient descent optimizer
            optimizer = tf.train.GradientDescentOptimizer(0.1)
            grads = optimizer.compute_gradients(self.loss)
            #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
            self.app = optimizer.apply_gradients(grads)

            self.init = tf.global_variables_initializer()
            
    def MLP(self, stacked_tensor, weights_input, weights_input_2, bias1, bias2):
        h1 = (tf.matmul(weights_input,stacked_tensor,transpose_a=True,transpose_b=True))
        h1 = tf.add(h1, bias1)
        h1 = tf.nn.tanh(h1)
        h2 = tf.matmul(weights_input_2,h1,transpose_a=True,transpose_b=False)
        h2 = tf.add(h2, bias2)
        h2 = tf.nn.sigmoid(h2)
        return tf.transpose(h2)
    
    
    def gradient_descent(self,total_loss):
        total_loss=tf.convert_to_tensor(total_loss)
        with tf.GradientTape(persistent=True) as tape:
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            print(variables)
        return total_loss, tape.gradient(tf.convert_to_tensor(total_loss), variables)
        
                                                   
    def train(self, sess, num_steps):
        self.init.run()
        print("Initailized")
        total_loss=0.0;
        for file in files[:1]:            
            file_data_frame, catch_words = fp.get_dataframe(file)
            sentences_list = get_file_sentences(file) #indexes/order preserved
            sentence_embeddings = sen2vec(sentences_list)
            sentence_ids_to_indices = np.array([int(x[1:]) for x in list(file_data_frame['Id'].values)])
            phrases_indices = np.array(list(file_dataframe['words'].values))
            phrases_label_matrix = np.array(list(file_dataframe['is_catchword'].values))
            doc_embedding = d2v_model.docvecs[file]
            feed_dict = {self.doc_embedding: doc_embedding, self.phrases_indices: phrases_indices, self.phrase_labels: phrases_label_matrix, self.sentence_indices:sentence_ids_to_indices, self.sentence_embeddings: sentence_embeddings}            
            #loss_val = sess.run([self.loss], feed_dict=feed_dict)  
            _,loss_val,outputs = sess.run([self.app,self.loss,self.sentence_values], feed_dict=feed_dict)

            print("Train Finished: loss_val=", loss_val)
            for i in range(len(outputs)):
                print(outputs[i])
            
        
    def load_embeddings(self,embedding_array):
        sess.run(self.embedding_init, feed_dict={self.embedding_placeholder: np.asarray(list(embedding_array.values()))})
        

    

In [41]:
def get_features(inputs):
    out=np.zeros((len(inputs),200));
    for i in range(len(inputs)):
        try:
            c = w2v[inputs[i]]
        except KeyError:
            c = np.zeros((1,200));
        out[i]=c;
    
    return np.transpose(out);

In [ ]:
graph = tf.Graph()

model = CatchPhraseExtractor(graph)


generating graph


In [ ]:
with tf.Session(graph=graph) as sess:
  
    #model.load_embeddings(embedding_array)    
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    #global_step = tf.train.get_or_create_global_step()
    for i in range(1):
        model.train(sess, 10)
    #optimizer.apply_gradients(zip(grads, model.variables), global_step)
        